In [303]:
import numpy as np
import pandas as pd
import scipy.linalg
from numpy.linalg import matrix_power

In [307]:
def dm_to_bloch_reg(rho):
    # rho is a density matrix
    state = dm_to_bloch_vector(rho)
    state = cartesian_to_spherical(state)

    # state is now (theta, phi, r)
    for i, intv in enumerate(thetas):
        if (state[0] in intv):
            theta_reg = i
    for i, intv in enumerate(phis):
        if (state[1] in intv):
            phi_reg = i
    for i, intv in enumerate(radii):
        if (state[2] in intv):
            r_reg = i

    if (theta_reg == 0):
        theta_reg = phi_reg = 0
    if (theta_reg == len(thetas)-1):
        theta_reg = len(thetas)-1
        phi_reg = len(phis)-1

    return (theta_reg, phi_reg, r_reg)

def dm_to_bloch_vector(rho):
    x = np.array([[0,1],[1,0]])
    y = np.array([[0,-1j],[1j,0]])
    z = np.array([[1,0],[0,-1]])

    # where rho is a density matrix
    return (np.trace(rho @ x).real, np.trace(rho @ y).real, np.trace(rho @ z).real)

def dm_to_polar_coords(rho):
    # rho is a density matrix
    return cartesian_to_spherical(dm_to_bloch_vector(rho))

def cartesian_to_spherical(state):
    x = state[0]
    y = state[1]
    z = state[2]

    r = min(np.sqrt(x**2 + y**2 + z**2), 1)
    theta = max(0, np.arccos(z/r))
    phi = np.arctan2(y.real, x.real)
    if (phi < 0): phi += 2*np.pi

    return (theta, phi, r)

def spherical_to_cartesian(state):
    theta = state[0]
    phi = state[1]
    r = state[2]

    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)

    if abs(x) < 0.0001:
        x = 0
    if abs(y) < 0.0001:
        y = 0
    if abs(z) < 0.0001:
        z = 0
    return (x, y, z)

def random_state_in_reg(reg):
    # where reg is a tuple specifying (theta, phi, radius)
    # returns a density matrix
    theta = np.random.uniform(thetas[reg[0]].left, thetas[reg[0]].right)

    # maybe consider the poles as one state
    if (reg[0] == 0 or reg == len(thetas)-1):
        phi = np.random.uniform(0, 2*np.pi)
    else:
        phi = np.random.uniform(phis[reg[1]].left, phis[reg[1]].right)
    # r = np.random.uniform(radii[reg[2]].left, radii[reg[2]].right)
    r = 1

    state = spherical_to_cartesian((theta, phi, r))
    rho = (np.eye(2) + state[0]*np.array([[0,1],[1,0]]) + state[1]*np.array([[0, -1j], [1j, 0]]) + state[2]*np.array([[1,0], [0,-1]]))/2
    return np.matrix(rho)

def generate_target_state(n):
    s = np.matrix([1, 0])
    rho = np.outer(s, s.H)
    
    ht = GATES[0] @ GATES[1]
    rho = matrix_power(ht, n) @ rho @ matrix_power(ht.H, n)
        
    return rho

def apply_operator(rho, op): # add noise
    return np.matrix(op @ rho @ op.H)

def dm_fidelity(rho, sigma):
    rho_sqrt = scipy.linalg.sqrtm(rho)
    return np.trace(scipy.linalg.sqrtm(rho_sqrt @ sigma @ rho_sqrt))**2

In [310]:
n = 10**8
k = 16

GATES = [
    np.matrix([[1, 1], [1, -1]]) / np.sqrt(2), # H
    np.matrix([[1, 0], [0, np.exp(1j * np.pi / 4)]]), # T
    np.matrix([[1, 0], [0, 1]]) # I
]

goal = generate_target_state(n=n)
thetas = np.array(pd.cut(np.linspace(0, np.pi, k), k, precision=10, include_lowest=True))
thetas[0] = pd.Interval(0, thetas[0].right, closed='both')
phis = np.array(pd.cut(np.linspace(0, 2*np.pi, k), k,  precision=10, include_lowest=True))
phis[0] = pd.Interval(0, phis[0].right, closed='both')
# rs = (1 - np.geomspace(1e-3, 1, k))[::-1]
# rs[0] = 0
# rs[-1] = 1
# radii = pd.cut(np.linspace(0, 1, 1), 1, precision=10, include_lowest=True)
radii = [pd.Interval(0, 1, closed='both')]
goal_reg = dm_to_bloch_reg(goal)

states = [(i, j, k) for i in range(len(thetas)) for j in range(len(phis)) for k in range(len(radii))]
values = np.zeros(len(thetas) * len(phis) * len(radii))

In [311]:
print(np.trace(goal@goal.H))
print(goal_reg)
# print(dm_to_bloch_vector(goal))
print(dm_to_polar_coords(goal))
print(goal)
print('0: ', np.trace(np.outer(np.matrix([1,0]), np.matrix([1,0]).H) @ goal))
print('1: ', np.trace(np.outer(np.matrix([0,1]), np.matrix([0,1]).H) @ goal))

(0.9999999678241469+0j)
(0, 0, 0)
(0.012715749262621721, 4.325563567729244, 0.9999999839120733)
[[ 9.99959562e-01+8.41340886e-17j -2.39844544e-03+5.88794102e-03j]
 [-2.39844544e-03-5.88794102e-03j  4.04220245e-05-1.91598853e-18j]]
0:  (0.9999595618875546+8.413408858487514e-17j)
1:  (4.042202451868746e-05-1.9159885266892274e-18j)


In [249]:
transitions = [np.zeros((len(states), len(states)), dtype=np.half) for i in range(len(GATES))]

# building transition matrices
for s in states:
    for i in range(100):
        state = random_state_in_reg(s)
        for j in range(len(GATES)):
            state_ind = states.index(dm_to_bloch_reg(state))
            n_state = apply_operator(state, GATES[j])
            n_state_ind = states.index(dm_to_bloch_reg(n_state))
            transitions[j][state_ind][n_state_ind] += 1

for i in range(len(GATES)):
    for j in range(len(states)):
        transitions[i][j] = np.nan_to_num(transitions[i][j] / sum(transitions[i][j]))

C:\Users\nfber\anaconda3\envs\qc\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


In [250]:
transitions[0][33]

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.06, 0.55, 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.01, 0.38, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.

In [251]:
def R(state, action):
    # pass
    if (state == goal_reg):
        return 1
    #     if (action <= len(GATES) - 2):
    #         return 0
    #     else:
    #         return 0.1 # to encourage using identity
    else:
        return 0

In [275]:
def policy_eval(policy, discount_factor=0.8, epsilon=0.000001):
    V_old = np.zeros(len(states))
    while True:
    # for i in range(1):
        V_new = np.zeros(len(states))
        delta = 0
        for s, _ in enumerate(states):
            v_fn = 0
            action_probs = policy[s]
            for a, _ in enumerate(GATES):
                p_trans = transitions[a][s]
                p_next_states = np.nonzero(transitions[a][s])[0]
                for next_s in p_next_states:
                    v_fn += action_probs[a] * p_trans[next_s] * (R(states[s], a) + discount_factor * V_old[next_s])
            delta = max(delta, abs(v_fn - V_old[s]))
            V_new[s] = v_fn
        V_old = V_new
        if(delta < epsilon):
            break
    # since technically the entire north/south pole is one state, copy (0, 0) and (k-1, k-1) over
    # won't ever be used, but it is needed for the visualization
    for i in range(1, len(phis)):
        V_old[i] = V_old[0]
        V_old[len(thetas)*len(phis)-1-i] = V_old[len(thetas)*len(phis)-1]
    return np.array(V_old)

In [276]:
def policy_improvement(policy_eval_fn=policy_eval, discount_factor=0.8):      
    def one_step_lookahead(s, V_old):
        actions = np.zeros(len(GATES))
        for a in range(len(GATES)):
            v_fn = 0
            p_trans = transitions[a][s]
            p_next_states = np.nonzero(transitions[a][s])[0]
            for next_s in p_next_states:
                v_fn += p_trans[next_s] * (R(states[s], a) + discount_factor * V_old[next_s])
            actions[a] = v_fn
        return actions
    policy = np.ones([len(states), len(GATES)]) / len(GATES)
    actions_values = np.zeros(len(GATES))
    
    while True:
        value_fn = policy_eval_fn(policy)
        policy_stable = True
        for s in range(len(states)):
            actions_values = one_step_lookahead(s, value_fn)
            best_action = np.argmax(actions_values)
            chosen_action = np.argmax(policy[s])
            if(best_action != chosen_action):
                policy_stable = False
            policy[s] = np.eye(len(GATES))[best_action]
        
        if(policy_stable):
            return policy, value_fn

In [312]:
policy, v = policy_improvement(policy_eval)

In [313]:
np.count_nonzero(v)

256

In [314]:
v

array([4.99999608, 4.99999608, 4.99999608, 4.99999608, 4.99999608,
       4.99999608, 4.99999608, 4.99999608, 4.99999608, 4.99999608,
       4.99999608, 4.99999608, 4.99999608, 4.99999608, 4.99999608,
       4.99999608, 1.4469387 , 1.5885998 , 1.8106244 , 1.98575073,
       1.90135308, 1.69260513, 1.11031356, 1.26798305, 1.38789293,
       1.58497979, 1.73486715, 1.98122572, 2.109929  , 1.81919591,
       1.15755017, 1.27087905, 1.00237587, 1.34489767, 1.25297082,
       1.68112306, 1.56621451, 0.70294847, 0.50468379, 0.47707264,
       0.56855683, 0.59634178, 0.71069701, 0.68858569, 0.68854497,
       0.86073309, 0.80189991, 1.07591735, 0.8641963 , 1.16382339,
       1.08024636, 1.45478021, 1.35030893, 0.53105246, 0.37600023,
       0.46620809, 0.47000126, 0.51204287, 0.58750256, 0.60318574,
       0.6069545 , 0.74484556, 0.71845869, 0.93105793, 0.77614403,
       1.01895252, 0.97018102, 1.27369164, 1.21272725, 0.42113178,
       0.34972009, 0.46296272, 0.43715109, 0.57870438, 0.54643

In [315]:
optimal_programs = []
for i in range(k):
    converged = False
    while not converged:
        s = random_state_in_reg((0, 0, 0))
        prog = []
        counter = 0
        while counter < 30:
            action = np.argmax(policy[states.index(dm_to_bloch_reg(s))])
            next_s = apply_operator(s, GATES[action])
            prog.append(action)
            # next_s = random_state_in_reg(dm_to_bloch_reg(next_s))
            s = next_s
            counter += 1
            if (dm_to_bloch_reg(s) == goal_reg):
                print('converged')
                converged = True
                break
        
    optimal_programs.append(prog)
optimal_programs

converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged
converged


[[1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1]]

In [316]:
for prog in optimal_programs:
    psi = np.matrix([1,0])
    rho = np.matrix(np.outer(psi, psi.H))

    for g in prog:
        rho = apply_operator(rho, GATES[g])
    print(prog, dm_fidelity(goal, rho))

[1] (0.9999595620244383+1.3688449738429297e-10j)
[1] (0.9999595620244383+1.3688449738429297e-10j)
[1] (0.9999595620244383+1.3688449738429297e-10j)
[1] (0.9999595620244383+1.3688449738429297e-10j)
[1] (0.9999595620244383+1.3688449738429297e-10j)
[1] (0.9999595620244383+1.3688449738429297e-10j)
[1] (0.9999595620244383+1.3688449738429297e-10j)
[1] (0.9999595620244383+1.3688449738429297e-10j)
[1] (0.9999595620244383+1.3688449738429297e-10j)
[1] (0.9999595620244383+1.3688449738429297e-10j)
[1] (0.9999595620244383+1.3688449738429297e-10j)
[1] (0.9999595620244383+1.3688449738429297e-10j)
[1] (0.9999595620244383+1.3688449738429297e-10j)
[1] (0.9999595620244383+1.3688449738429297e-10j)
[1] (0.9999595620244383+1.3688449738429297e-10j)
[1] (0.9999595620244383+1.3688449738429297e-10j)
